### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [1]:
# environment file
import platform
whereami = platform.system()
print(f"I'm on {whereami}")
if whereami == 'Darwin':  # Mac
    environment_file = 'Reacher_one.app.app'
else:
    environment_file = 'Reacher_Linux_NoVis/Reacher.x86_64'
    
print(f'Using {environment_file}')

I'm on Darwin
Using Reacher_one.app.app


In [2]:
env = None
import torch
print(torch.torch.cuda.is_available())
!make Reacher_Linux_NoVis

False
make: `Reacher_Linux_NoVis' is up to date.


In [3]:
import sys
sys.path.append('../')
from unityagents import UnityEnvironment

from continuous import logger
from continuous.agents import DDPG_Agent
from continuous.environment import UnityAdapter
from continuous.problem import moving_average, loggable, moving_average_cross

import numpy as np

import mlflow
mlflow.set_tracking_uri('file:../mlruns')
mlflow.end_run()


logger.info('Reacher environment')
if env is None:
    env = UnityAdapter(
        UnityEnvironment(
            file_name=environment_file,
            no_graphics=True,
        ),
        name="continuous_1")
# state_size=33, action_size=4,
agent = DDPG_Agent(
    env, seed=0,
    is_finished_function=[loggable(moving_average_cross, 100, 200), loggable(moving_average, 100, 30)],
    is_solved=loggable(moving_average, 100, 13),
    epsilon_update=loggable(lambda x: x),
    update_every=10,
    update_times=20
    )

mlflow.set_experiment(agent.environment.name)
mlflow.start_run()

scores = agent.train(
    max_training_episodes=5)
# env.close()
logger.info(scores)

    # there are a LOT of meta parameters


2020-05-09 17:14:30,519|INFO|continuous|Reacher environment
INFO:continuous:Reacher environment
INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
2020-05-09 17:14:31,817|INFO|continuous|Starting training for DDPG Agent
INFO:continuous:Starting training for DDPG Agent
2020-05-09 17:14:31,990|WARNING|continuous|Model class method not implemented. Moving on regardless.
2020-05-09 17:15:21,051|DEBUG|continuous|Episode     1: 0.100000
DEBUG:co

In [4]:
agent.train(
    max_training_episodes=5,)

2020-05-09 17:18:06,847|INFO|continuous|Starting training for DDPG Agent
INFO:continuous:Starting training for DDPG Agent
2020-05-09 17:18:07,028|WARNING|continuous|Model class method not implemented. Moving on regardless.
2020-05-09 17:18:49,437|DEBUG|continuous|Episode     6: 0.260000
DEBUG:continuous:Episode     6: 0.260000
2020-05-09 17:19:28,937|DEBUG|continuous|Episode     7: 0.450000
DEBUG:continuous:Episode     7: 0.450000
2020-05-09 17:20:08,105|DEBUG|continuous|Episode     8: 0.210000
DEBUG:continuous:Episode     8: 0.210000
2020-05-09 17:20:46,382|DEBUG|continuous|Episode     9: 0.130000
DEBUG:continuous:Episode     9: 0.130000
2020-05-09 17:21:26,368|DEBUG|continuous|Episode    10: 0.200000
DEBUG:continuous:Episode    10: 0.200000
2020-05-09 17:21:26,414|INFO|continuous|DDPG Agent trained finished after 10 episodes
INFO:continuous:DDPG Agent trained finished after 10 episodes
2020-05-09 17:21:26,476|DEBUG|continuous|Saving to model_final.pkl
DEBUG:continuous:Saving to model

False

In [5]:
mlflow.end_run()